In [2]:
# General stuff
import os
import re
import time
import warnings
import math
import sys
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt
import nibabel as nib
from subprocess import Popen, PIPE
from os.path import join as opj
from IPython.core.debugger import Tracer
from nilearn.decomposition import CanICA,DictLearning
from sklearn.decomposition import DictionaryLearning

# sklearn stuff
from sklearn.decomposition import TruncatedSVD,FastICA,SparsePCA,MiniBatchSparsePCA
from sklearn.utils.extmath import randomized_svd
from sklearn.linear_model import Ridge
from sklearn.decomposition import dict_learning_online
from sklearn.cross_decomposition import CCA
from sklearn.preprocessing import scale
import sklearn.metrics.cluster as metrics
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster.hierarchical import _hc_cut # Internal function to cut ward tree, helps speed up things a lot
from sklearn.utils import resample
from sklearn.model_selection import KFold

# personal functions
from importlib import reload
import fsutils as fs
reload(fs)

if os.path.exists('/data1/vbeliveau/'):
    # On NRU server
    cluster_code='/data1/vbeliveau/5HT_clustering'
    cluster_data='/data1/vbeliveau/5HT_clustering_data'
    subjects_dir='/usr/local/nru/freesurfer/subjects'
elif os.path.exists('C:/Users/vbeliveau/'):
    # On laptop
    cluster_code='C:/Users/vbeliveau/Documents/GitHub/5HT_clustering'
    cluster_data='C:/Users/vbeliveau/Documents/5HT_clustering_data'
    subjects_dir='C:/Users/vbeliveau/Downloads/'
else:
    raise ValueError('Unknown location')
    
# Load local version of nilearn
if os.path.exists('/data1/vbeliveau/'):
    sys.path.append('/data1/vbeliveau/nilearn')
    import mynilearn.decomposition as dcm 
    reload(dcm)
elif os.path.exists('C:/Users/vbeliveau/'):
    sys.path.append('C:/Users/vbeliveau/Downloads/nilearn-master')
    import mynilearn.decomposition as dcm
    reload(dcm)
else:
    raise ValueError('Unknown location')
    
# Analysis directories
surf_data=opj(cluster_data,'surf_data')
fs.assert_dir(surf_data)
BPnd_data=opj(cluster_data,'BPnd')
fs.assert_dir(BPnd_data)

info_path=opj(cluster_code,'info_alltracers_base_healthy_hrrt.mat')
hemi_type=['lh','rh']

info=sio.loadmat(info_path)
tracers=['cumi','dasb','sb','az','C36']
petID=[item for subl in info['petID'] for subsubl in subl for item in subsubl]

In [ ]:
# define MSMdir
msmDir_code = '/data1/vbeliveau/software/MSM_HOCR_v1';
msmDir_data = '/data1/vbeliveau/5HT_clustering_data/surf_data/';
msmDir_analysis = '/data1/Ganz/Vincent'; # change this to a place we both can write


# load data
tracers=['cumi'] #,'dasb','sb','az','C36']
hemi_type=['lh'] #,'rh']
targ='ico'
sm='5'

data=np.empty(len(tracers),dtype=object)
for tracer,nt in zip(tracers,np.arange(0,len(tracers))):
    for hemi in hemi_type:
        fmask=opj(surf_data,'mask.'+targ+'.'+hemi) # UPDATE NAMES!!!
        fname=opj(BPnd_data,'maps',tracer,'mrtm2.nopvc.' + 
                  targ + '.' + hemi + '.sm' + sm + '.nii.gz') # UPDATE NAMES!!!
        if data[nt] is None:
            data[nt]=fs.save_surf_data(fname,mask=fmask,text) #need Vincent's flag
        else:
            data[nt]=np.vstack((data[nt],fs.load_surf_data(fname,mask=fmask)))

# run msms command
# define config file, see https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/MSM/UserGuide for details
configFile = [msmDir_analysis,'msm_sulc_depth.config']

# Procedure 
# 1) use sulc aligned functional data on sphere and align the single subject once given 
for tracer,nt in zip(tracers,np.arange(0,len(tracers))):
    subjlist=petID[np.array([re.search('^'+tracer,x) is not None for x in petID])]
    for hemi in hemi_type:
        for subj in subjlist:
            log_file=open(opj(msmDir_analysis,'msm'+subj+'.'+ hemi +'.log'),'w')
            p=Popen(['./msm','--indata=',msmDir_data,'maps',tracer,'mrtm2.nopvc.',
                              targ,'.',hemi,'.sm',sm,'.asc','--refdata=',msmDir_data,'means',tracer,'mrtm2.nopvc.',
                              targ,'.',hemi,'.sm',sm,'.asc','--inmesh=/usr/local/nru/freesurfer/subjects/fsaverage/surf/lh.sphere.shape.gii',
                              '--out=',msmsDir_analysis,'--conf',configFile,'-p','-v',],stdout=log_file, stderr=log_file)
            # UPDATE NAMES!!!
            p.communicate()
            log_file.close()


# 2) transform functional data on each subject and redefine an atlas based on the transformed subjects



# 3) rerun msm
